In [1]:
!pip install -q flwr[simulation] torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 9.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.3 which is incompatible.


In [1]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10
import time
import flwr as fl

DEVICE = torch.device("cuda")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

Training on cuda using PyTorch 1.13.1 and Flower 1.4.0


In [2]:
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
   process = psutil.Process(os.getpid())
   print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
   print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 125.7 GB  | Proc size: 303.6 MB
GPU RAM Free: 15758MB | Used: 410MB | Util   3% | Total 16376MB


In [3]:
NUM_CLIENTS = 10


def load_datasets(num_clients: int):
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = CIFAR10("./dataset", train=True, download=True, transform=transform)
    testset = CIFAR10("./dataset", train=False, download=True, transform=transform)

    # Split training set into `num_clients` partitions to simulate different local datasets
    partition_size = len(trainset) // num_clients
    lengths = [partition_size] * num_clients
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=32, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=32))
    testloader = DataLoader(testset, batch_size=32)
    return trainloaders, valloaders, testloader


trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):  # Use the passed 'epochs' variable here
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss.item()  # Make sure to call .item() to get the scalar value
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch}: train loss {epoch_loss:.6f}, accuracy {epoch_acc:.6f}")



def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

In [5]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        epochs = config.get("epochs", 1)
        start_time = time.time()  # Start time measurement
        train(self.net, self.trainloader, epochs)
        training_time = time.time() - start_time  # Calculate duration
        print(f"Training time for Client {self.cid}: {training_time:.2f} seconds")
        return get_parameters(self.net), len(self.trainloader), {"training_time": training_time}



    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

In [6]:
from typing import Callable, Union

from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    MetricsAggregationFn,
    NDArrays,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy.aggregate import aggregate, weighted_loss_avg


class FedCustom(fl.server.strategy.Strategy):
    def __init__(
        self,
        fraction_fit: float = 1.0,
        fraction_evaluate: float = 1.0,
        min_fit_clients: int = 2,
        min_evaluate_clients: int = 2,
        min_available_clients: int = 2,
    ) -> None:
        super().__init__()
        self.fraction_fit = fraction_fit
        self.fraction_evaluate = fraction_evaluate
        self.min_fit_clients = min_fit_clients
        self.min_evaluate_clients = min_evaluate_clients
        self.min_available_clients = min_available_clients
        self.client_training_times = {}
    def __repr__(self) -> str:
        return "FedCustom"

    def initialize_parameters(
        self, client_manager: ClientManager
    ) -> Optional[Parameters]:
        """Initialize global model parameters."""
        net = Net()
        ndarrays = get_parameters(net)
        return fl.common.ndarrays_to_parameters(ndarrays)

    def configure_fit(self, server_round: int, parameters: Parameters, client_manager: ClientManager):
        sample_size, min_num_clients = self.num_fit_clients(client_manager.num_available())
        clients = client_manager.sample(num_clients=sample_size, min_num_clients=min_num_clients)
        epochs_sc = 5
        epochs_hl = 3

        standard_config = {"lr": 0.001, "epochs": epochs_sc}
        higher_lr_config = {"lr": 0.0001, "epochs": epochs_hl}
        fit_configurations = []

        for client in clients:
            # Choose config based on the previous training time
            last_time = self.client_training_times.get(client.cid, 0)  # Default to 0 if no time recorded
            print(f"This is the last time {last_time}")
            


            config_to_use = standard_config if last_time < 13.8 else higher_lr_config
            fit_configurations.append((client, FitIns(parameters, config_to_use)))

        return fit_configurations

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        """Aggregate fit results using weighted average."""
        for client, fit_res in results:
            # Update training times for each client
            self.client_training_times[client.cid] = fit_res.metrics.get("training_time", 0)
        weights_results = [
            (parameters_to_ndarrays(fit_res.parameters), fit_res.num_examples)
            for _, fit_res in results
        ]
        parameters_aggregated = ndarrays_to_parameters(aggregate(weights_results))
        metrics_aggregated = {}
        return parameters_aggregated, metrics_aggregated


    def configure_evaluate(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, EvaluateIns]]:
        """Configure the next round of evaluation."""
        if self.fraction_evaluate == 0.0:
            return []
        config = {}
        evaluate_ins = EvaluateIns(parameters, config)

        # Sample clients
        sample_size, min_num_clients = self.num_evaluation_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )

        # Return client/config pairs
        return [(client, evaluate_ins) for client in clients]

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[Union[Tuple[ClientProxy, EvaluateRes], BaseException]],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        """Aggregate evaluation losses using weighted average."""

        if not results:
            return None, {}

        loss_aggregated = weighted_loss_avg(
            [
                (evaluate_res.num_examples, evaluate_res.loss)
                for _, evaluate_res in results
            ]
        )
        metrics_aggregated = {}
        return loss_aggregated, metrics_aggregated

    def evaluate(
        self, server_round: int, parameters: Parameters
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        """Evaluate global model parameters using an evalua
        tion function."""

        # Let's assume we won't perform the global model evaluation on the server side.
        return None

    def num_fit_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Return sample size and required number of clients."""
        num_clients = int(num_available_clients * self.fraction_fit)
        return max(num_clients, self.min_fit_clients), self.min_available_clients

    def num_evaluation_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Use a fraction of available clients for evaluation."""
        num_clients = int(num_available_clients * self.fraction_evaluate)
        return max(num_clients, self.min_evaluate_clients), self.min_available_clients

In [7]:
if DEVICE.type == "cuda":
    # Use a single client to train the global model
    client_resources = {"num_gpus": .25, "num_cpus": 2} 

In [ ]:

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=10,
    config=fl.server.ServerConfig(num_rounds=3),
    strategy=FedCustom(),  # <-- pass the new strategy here
    client_resources=client_resources,

)

INFO flwr 2024-06-27 13:44:10,502 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2024-06-27 13:44:14,913	INFO worker.py:1752 -- Started a local Ray instance.
INFO flwr 2024-06-27 13:44:16,893 | app.py:180 | Flower VCE: Ray initialized with resources: {'accelerator_type:RTX': 1.0, 'GPU': 1.0, 'node:__internal_head__': 1.0, 'memory': 77620241408.0, 'object_store_memory': 37551532032.0, 'node:127.0.0.1': 1.0, 'CPU': 32.0}
INFO flwr 2024-06-27 13:44:16,895 | server.py:86 | Initializing global parameters
INFO flwr 2024-06-27 13:44:16,902 | server.py:269 | Using initial parameters provided by strategy
INFO flwr 2024-06-27 13:44:16,904 | server.py:88 | Evaluating initial parameters
INFO flwr 2024-06-27 13:44:16,905 | server.py:101 | FL starting
DEBUG flwr 2024-06-27 13:44:16,906 | server.py:218 | fit_round 1: strategy sampled 10 clients (out of 10)


This is the last time 0
This is the last time 0
This is the last time 0
This is the last time 0
This is the last time 0
This is the last time 0
This is the last time 0
This is the last time 0
This is the last time 0
This is the last time 0
(launch_and_fit pid=21420) [Client 5] fit, config: {'lr': 0.001, 'epochs': 5}
(launch_and_fit pid=10780) Epoch 0: train loss 0.064418, accuracy 0.233778
(launch_and_fit pid=43732) [Client 7] fit, config: {'lr': 0.001, 'epochs': 5} [repeated 3x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(launch_and_fit pid=42984) Epoch 2: train loss 0.051134, accuracy 0.396667 [repeated 9x across cluster]
(launch_and_fit pid=10780) Training time for Client 1: 21.21 seconds
(launch_and_fit pid=43732) Epoch 4: train loss 0.045833, accuracy 0.472444 [repeated 8x across cluster]
(launch_and_fit pid=7400) [C

DEBUG flwr 2024-06-27 13:45:46,823 | server.py:232 | fit_round 1 received 10 results and 0 failures
DEBUG flwr 2024-06-27 13:45:46,880 | server.py:168 | evaluate_round 1: strategy sampled 10 clients (out of 10)


(launch_and_fit pid=6040) Training time for Client 6: 20.80 seconds
(launch_and_fit pid=6040) Epoch 4: train loss 0.047430, accuracy 0.435333 [repeated 4x across cluster]
(launch_and_evaluate pid=31824) [Client 5] evaluate, config: {}
(launch_and_fit pid=27796) Training time for Client 3: 20.71 seconds
(launch_and_fit pid=27796) Epoch 4: train loss 0.047805, accuracy 0.432000
(launch_and_evaluate pid=42988) [Client 0] evaluate, config: {} [repeated 4x across cluster]
(launch_and_evaluate pid=7044) [Client 2] evaluate, config: {} [repeated 2x across cluster]
(launch_and_evaluate pid=24988) [Client 3] evaluate, config: {}
(launch_and_evaluate pid=41232) [Client 1] evaluate, config: {}


In [12]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=5)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        print(f"Client {self.cid} loss {loss}")
        print(f"Client {self.cid} accuracy {accuracy}")
        
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE) #Load Model from here
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

In [ ]:
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=10,
    config=fl.server.ServerConfig(num_rounds=3),
    client_resources=client_resources,
)

INFO flwr 2024-05-29 11:36:33,564 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)


INFO flwr 2024-05-29 11:36:40,084 | app.py:180 | Flower VCE: Ray initialized with resources: {'GPU': 1.0, 'CPU': 32.0, 'memory': 61953181082.0, 'object_store_memory': 30837077606.0, 'node:127.0.0.1': 1.0}
INFO flwr 2024-05-29 11:36:40,087 | server.py:86 | Initializing global parameters
INFO flwr 2024-05-29 11:36:40,089 | server.py:273 | Requesting initial parameters from one random client
INFO flwr 2024-05-29 11:36:44,026 | server.py:277 | Received initial parameters from one random client
INFO flwr 2024-05-29 11:36:44,027 | server.py:88 | Evaluating initial parameters
INFO flwr 2024-05-29 11:36:44,029 | server.py:101 | FL starting
DEBUG flwr 2024-05-29 11:36:44,031 | server.py:218 | fit_round 1: strategy sampled 10 clients (out of 10)


 pid=27924) [Client 3] get_parameters
 pid=27924) [Client 8] fit, config: {}
 pid=27000) [Client 9] fit, config: {}
 pid=26956) [Client 5] fit, config: {}
 pid=19244) [Client 3] fit, config: {}
 pid=27924) Epoch 0: train loss 0.064283, accuracy 0.244000
 pid=27924) Epoch 1: train loss 0.056345, accuracy 0.335556
 pid=27924) Epoch 2: train loss 0.052596, accuracy 0.383111
 pid=27000) Epoch 0: train loss 0.065390, accuracy 0.219333
 pid=26956) Epoch 0: train loss 0.065386, accuracy 0.222667
 pid=27924) Epoch 3: train loss 0.049705, accuracy 0.431111
 pid=19244) Epoch 0: train loss 0.064883, accuracy 0.228667
 pid=27000) Epoch 1: train loss 0.057237, accuracy 0.326667
 pid=26956) Epoch 1: train loss 0.057874, accuracy 0.322444
 pid=27924) Epoch 4: train loss 0.047198, accuracy 0.454667
 pid=27924) [Client 1] fit, config: {}
 pid=19244) Epoch 1: train loss 0.056371, accuracy 0.348444
 pid=27000) Epoch 2: train loss 0.053247, accuracy 0.374667
 pid=26956) Epoch 2: train loss 0.054083, accur

DEBUG flwr 2024-05-29 11:37:45,189 | server.py:232 | fit_round 1 received 10 results and 0 failures
WARNING flwr 2024-05-29 11:37:45,233 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2024-05-29 11:37:45,235 | server.py:168 | evaluate_round 1: strategy sampled 10 clients (out of 10)


 pid=27000) Epoch 4: train loss 0.048110, accuracy 0.436444
 pid=27000) [Client 4] evaluate, config: {}
 pid=27000) Client 4 loss 0.057416656255722044
 pid=27000) Client 4 accuracy 0.348
 pid=27924) [Client 8] evaluate, config: {}
 pid=27924) Client 8 loss 0.05749241328239441
 pid=27924) Client 8 accuracy 0.394
 pid=27924) [Client 6] evaluate, config: {}
 pid=27000) [Client 2] evaluate, config: {}
 pid=27924) Client 6 loss 0.05749799847602844
 pid=27924) Client 6 accuracy 0.348
 pid=27000) Client 2 loss 0.058668370962142946
 pid=27000) Client 2 accuracy 0.342
 pid=27000) [Client 9] evaluate, config: {}
 pid=27000) Client 9 loss 0.05861459255218506
 pid=27000) Client 9 accuracy 0.36
 pid=27000) [Client 1] evaluate, config: {}
 pid=27000) Client 1 loss 0.057283300876617434
 pid=27000) Client 1 accuracy 0.376


...

 pid=27000) Epoch 4: train loss 0.033772, accuracy 0.621778
 pid=27000) [Client 7] evaluate, config: {}
 pid=27000) Client 7 loss 0.04311247169971466
 pid=27000) Client 7 accuracy 0.518
 pid=27000) [Client 2] evaluate, config: {}
 pid=27924) [Client 5] evaluate, config: {}
 pid=27000) Client 2 loss 0.04556352210044861
 pid=27000) Client 2 accuracy 0.5
 pid=26956) [Client 3] evaluate, config: {}
 pid=27924) Client 5 loss 0.0403660671710968
 pid=27924) Client 5 accuracy 0.516
 pid=26956) Client 3 loss 0.041824417114257814
 pid=26956) Client 3 accuracy 0.554
 pid=26956) [Client 6] evaluate, config: {}
 pid=26956) Client 6 loss 0.04151818835735321
 pid=26956) Client 6 accuracy 0.526
 pid=26956) [Client 0] evaluate, config: {}
 pid=26956) Client 0 loss 0.04397455382347107
 pid=26956) Client 0 accuracy 0.51
 pid=27924) [Client 9] evaluate, config: {}
 pid=27000) [Client 1] evaluate, config: {}
 pid=26956) [Client 4] evaluate, config: {}
 pid=19244) [Client 8] evaluate, config: {}


DEBUG flwr 2024-05-29 11:40:01,820 | server.py:182 | evaluate_round 3 received 10 results and 0 failures
INFO flwr 2024-05-29 11:40:01,822 | server.py:147 | FL finished in 197.79143010000007
INFO flwr 2024-05-29 11:40:01,824 | app.py:218 | app_fit: losses_distributed [(1, 0.05757532110214233), (2, 0.045601885247230536), (3, 0.042524653851985936)]
INFO flwr 2024-05-29 11:40:01,825 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2024-05-29 11:40:01,826 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2024-05-29 11:40:01,827 | app.py:221 | app_fit: losses_centralized []
INFO flwr 2024-05-29 11:40:01,827 | app.py:222 | app_fit: metrics_centralized {}


History (loss, distributed):
	round 1: 0.05757532110214233
	round 2: 0.045601885247230536
	round 3: 0.042524653851985936

 pid=27924) Client 9 loss 0.04389840006828308
 pid=27924) Client 9 accuracy 0.516
 pid=27000) Client 1 loss 0.04200864565372467
 pid=27000) Client 1 accuracy 0.526
 pid=26956) Client 4 loss 0.042090866446495054
 pid=26956) Client 4 accuracy 0.498
 pid=19244) Client 8 loss 0.040889406085014346
 pid=19244) Client 8 accuracy 0.53


Exception in thread 2024-05-29 18:33:30,550	ERROR import_thread.py:85 -- ImportThread: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "Stream removed"
	debug_error_string = "{"created":"@1716987810.545000000","description":"Error received from peer ipv4:127.0.0.1:62225","file":"src/core/lib/surface/call.cc","file_line":1075,"grpc_message":"Stream removed","grpc_status":2}"
>
ray_listen_error_messages:
Traceback (most recent call last):
  File "c:\Users\Admin\anaconda3\envs\flwrpytorch\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\Users\Admin\anaconda3\envs\flwrpytorch\lib\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\Admin\anaconda3\envs\flwrpytorch\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\Admin\anaconda3\envs\flwrpytorch\lib\site-packages\ray\worker.py", line 1311, in listen_error_messag